<a href="https://colab.research.google.com/github/BhavanaLalwani/Text-Sentiment-Analysis/blob/master/Project_F_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)

[================================================--] 97.3% 1617.2/1662.8MB downloaded

In [0]:
# Upgrade TensorFlow and Keras
# !pip install --upgrade tensorflow
# !pip install --upgrade keras

# Standard imports
import numpy as np
import pandas as pd
import csv
import argparse
import os
import json
import random
import math

# Keras imports
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, Input
from keras.layers import Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.models import Model, Sequential
from keras.utils import to_categorical
from keras.layers import Conv1D, MaxPooling1D, Bidirectional, LSTM, Dropout, GlobalMaxPooling1D
from keras.initializers import Constant
from keras import backend as K
import keras


from nltk.corpus import stopwords
import nltk
BASE_DIR = 'drive/My Drive/ECE542-ProjectF/'
DATA_DIR = BASE_DIR + 'data/'
GLOVE_DIR = 'drive/My Drive/ECE542-ProjectF/data/GloVe'
OUT_DIR = 'drive/My Drive/ECE542-ProjectF/output'
ISEAR_PATH = DATA_DIR + 'isear/isear_clean.csv'
W2V_PATH = DATA_DIR + 'w2v/GoogleNews-vectors-negative300.bin'
# TEXT_DATA_DIR = os.path.join(BASE_DIR, '20_newsgroup')
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100

ISEAR_SHUFFLE_SEED = 62
KAGGLE_SHUFFLE_SEED = 43

ISEAR_TEST_SPLIT = 0.1
ISEAR_VALIDATION_SPLIT = 0.1
KAGGLE_VALIDATION_SPLIT = 0.2


Using TensorFlow backend.


In [0]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score

import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

def split_data(data, labels, val_split, test_split, seed):
  # Shuffle our dataset. Using a known seed allows us to get the same result 
  # each time we run the script.
  indices = np.arange(data.shape[0])
  random.Random(seed).shuffle(indices)

  data = data[indices]
  labels = labels[indices]

  # Determine number of samples in train/val/test
  num_val = math.floor(data.shape[0] * val_split)
  num_test = math.floor(data.shape[0] * test_split)

  # Split the data into the test, val, and train subsets.
  x_test = data[:num_test]
  y_test = labels[:num_test]

  x_val = data[num_test:num_val + num_test]
  y_val = labels[num_test:num_val + num_test]

  x_train = data[num_val + num_test:]
  y_train = labels[num_val + num_test:]

  # Return a dictionary of the data with train/val/test and x/y.
  return {
      'train': {
          'x': x_train, 'y': y_train
      },
      'val': {
          'x': x_val, 'y': y_val
      },
      'test': {
          'x': x_test, 'y': y_test
      }
  }


In [0]:
data=pandas.read_csv(ISEAR_PATH)
data=pandas.DataFrame(data)
data.columns=['label','text']

x_train, x_val, y_train, y_val = model_selection.train_test_split(data['text'], data['label'])

dataset_test = split_data(
    data['text'],
    data['label'],
    ISEAR_VALIDATION_SPLIT, 
    ISEAR_TEST_SPLIT, 
    ISEAR_SHUFFLE_SEED
)


x_train = dataset_test['train']['x']
y_train = dataset_test['train']['y']
x_val = dataset_test['val']['x']
y_val = dataset_test['val']['y']
x_test = dataset_test['test']['x']
y_test = dataset_test['test']['y']

def train_model(classifier, feature_vector_train, label, feature_vector_valid, feature_vector_test, is_neural_net=False):
  # fit the training dataset on the classifier
  classifier.fit(feature_vector_train, label)
  # predict the labels on validation dataset
  predictions = classifier.predict(feature_vector_valid)
  test_predictions = classifier.predict(feature_vector_test)
  if is_neural_net:
    predictions = predictions.argmax(axis=-1)
  
  return predictions, test_predictions


tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1,1), max_features=10000)
tfidf_vect_ngram.fit(data['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(x_train)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(x_val)
xtest_tfidf_ngram =  tfidf_vect_ngram.transform(x_test)


# label encode the target variable 
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_val = encoder.fit_transform(y_val)
y_test = encoder.fit_transform(y_test)

val_pred, test_pred = train_model(svm.SVC(), xtrain_tfidf_ngram, y_train, xvalid_tfidf_ngram, xtest_tfidf_ngram)

print('Validation Metrics:')
report = classification_report(y_val, val_pred)
print(report)

print('Test Metrics:')
report = classification_report(y_test, test_pred, digits=3)
print(report)
print('F1: {0:0.3f}'.format(f1_score(y_test, test_pred, average='macro')))
print('Precision: {0:0.3f}'.format(precision_score(y_test, test_pred, average='macro')))
print('Recall: {0:0.3f}'.format(recall_score(y_test, test_pred, average='macro')))

Validation Metrics:
              precision    recall  f1-score   support

           0       0.47      0.60      0.53       108
           1       0.57      0.66      0.62       104
           2       0.75      0.72      0.74       108
           3       0.45      0.48      0.47       102
           4       0.77      0.73      0.75       111
           5       0.74      0.58      0.65       114
           6       0.48      0.40      0.44        99

    accuracy                           0.60       746
   macro avg       0.61      0.60      0.60       746
weighted avg       0.61      0.60      0.60       746

Test Metrics:
              precision    recall  f1-score   support

           0      0.401     0.551     0.465       107
           1      0.537     0.607     0.570       107
           2      0.694     0.648     0.670       105
           3      0.550     0.487     0.516       113
           4      0.663     0.704     0.683        98
           5      0.681     0.549     0.608 